In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tdqm
!pip install ipywidgets==7.7.1

  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1322 sha256=d0e6b6ed1053c846a2bba31b1f0297580659e712ce5c38d9ce31a69ff2f981f3
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.0 MB/s eta 0:00:00


In [3]:
import requests
from tqdm import tqdm
import time

# URL to fetch data from PubMed
output_file_path = "/content/drive/MyDrive/my_files/PUBMED_IDs_Part_5B.txt"
failed_pmid_file_path = "/content/drive/MyDrive/my_files/Failed_PMIDs_Part_1.txt"
pmid_list = set()

with open(failed_pmid_file_path, 'w', encoding='utf-8') as failed_file:
    # Initialize tqdm to track progress
    with open('/content/drive/MyDrive/my_files/bigpmid_triples_part5B.txt', 'r', encoding='latin-1') as infile:
        lines = infile.readlines()
        total_lines = len(lines)
        progress_bar = tqdm(total=total_lines)

        # Set to store unique PMIDs
        unique_pmids = set()

        start_time = time.time()
        for line in lines:
            rela_ids = []
            rela = eval(line)  # Convert the string representation of tuple to a tuple
            url = f"https://www.ncbi.nlm.nih.gov/research/pubtator3-api/search/?text={rela[0]} AND {rela[1]}"
            #print(f'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa')
            response = requests.get(url)
            if response.status_code == 200:
                result = response.json()
                if result['results']:  # Check if results exist
                    #print(f"{result['results'][0]['pmid']}aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
                    for item in result['results']:
                        pubmed_id = item['pmid']
                        if pubmed_id not in unique_pmids:
                            # Check if the text contains more than one word and is not None
                            pmid_list.add(pubmed_id)
                            unique_pmids.add(pubmed_id)
            else:
                print(f"Failed to retrieve pmid for Relationship: {rela}")
                failed_file.write(f'{rela}\n')

            # Periodically dump data to text file
            if len(pmid_list) >= 50:
                with open(output_file_path, 'a', encoding='utf-8') as outfile:
                    for pmid in pmid_list:
                        outfile.write(str(pmid) + '\n')
                pmid_list.clear()

            # Update progress bar
            progress_bar.update(1)

            # Limit requests to 3 per second
            if progress_bar.n % 3 == 0:
                elapsed_time = time.time() - start_time
                if elapsed_time < 1:
                    time.sleep(1 - elapsed_time)
                start_time = time.time()

        progress_bar.close()

# Write remaining PMIDs to the output file
with open(output_file_path, 'a', encoding='utf-8') as outfile:
    for pmid in pmid_list:
        outfile.write(str(pmid) + '\n')


  7%|▋         | 3314/48652 [48:52<10:39:59,  1.18it/s]

ConnectTimeout: HTTPSConnectionPool(host='www.ncbi.nlm.nih.gov', port=443): Max retries exceeded with url: /research/pubtator3-api/search/?text=(E)-cinnamoyl-CoA%20AND%20Evergreen%20huckleberry (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7e10a2748070>, 'Connection to www.ncbi.nlm.nih.gov timed out. (connect timeout=None)'))

In [ ]:
# Function to read PUBMED IDs from a file and store them in a set
              # def read_pubmed_ids(filename):
              #     pubmed_ids = set()
              #     with open(filename, 'r', encoding='utf-8') as file:
              #         for line in file:
              #             pubmed_ids.add(line.strip())
              #     return pubmed_ids

              # # Paths to the input files
              # file1_path = "/content/drive/MyDrive/my_files/Merged_PUBMED_IDs.txt"
              # file2_path = "/content/drive/MyDrive/my_files/PUBMED_ID_CHEMICALNAMES_2.txt"

              # # Read PUBMED IDs from both files
              # pubmed_ids_1 = read_pubmed_ids(file1_path)
              # pubmed_ids_2 = read_pubmed_ids(file2_path)

              # # Merge the sets to remove duplicates
              # merged_pubmed_ids = pubmed_ids_1.union(pubmed_ids_2)

              # # Write the unique PUBMED IDs to a new file
              # output_file_path = "/content/drive/MyDrive/my_files/PUBMED_IDs_THAT_NAV_LOVES.txt"
              # with open(output_file_path, 'w', encoding='utf-8') as outfile:
              #     for pubmed_id in merged_pubmed_ids:
              #         outfile.write(pubmed_id + '\n')

              # print("Merged PUBMED IDs written to:", output_file_path)
